In [2]:
# ---------- pair writer -------------------------------------------------------
def write_pair(pair_id: int,
               out_root: Path,
               *,
               ref_rgb_tok: np.ndarray,
               ref_coords_tok: np.ndarray,
               ref_caption: str,
               next_rgb_tok: np.ndarray,
               next_coords_tok: np.ndarray):
    pid = f"{pair_id:06d}"

    # check if this pair was already written
    marker = out_root/"tok_rgb"/pid/"tok_rgb.npy"
    if marker.exists():
        print(f"⏭  Skipping {pid} (already exists)")
        return False

    for modal in ("tok_rgb", "coords", "captions",
                  "tok_rgb_next", "coords_next"):
        (out_root / modal / pid).mkdir(parents=True, exist_ok=True)

    np.save(out_root/"tok_rgb"     /pid/"tok_rgb.npy",      ref_rgb_tok)
    np.save(out_root/"coords"      /pid/"coords.npy",       ref_coords_tok)
    with open(out_root/"captions"  /pid/"captions.json", "w") as fp:
        json.dump({"caption": ref_caption}, fp)

    np.save(out_root/"tok_rgb_next"/pid/"tok_rgb_next.npy", next_rgb_tok)
    np.save(out_root/"coords_next" /pid/"coords_next.npy",  next_coords_tok)
    return True


In [ ]:
# %% ------------------------------ Cell 1: library ------------------------------
"""
Penn-Action → fourM preprocessing helpers
========================================
Execute this cell once.  It defines functions but does **no I/O** until you call
`run(...)` from Cell 2.

**Output schema (per video)**
```
<OUTPUT_ROOT>/
  tok_rgb/        <video>/<ref>.npy          – Cosmos tokens of reference frame
  coords/         <video>/<ref>.npy          – 3×13 (x_tok, y_tok, vis)
  captions/       <video>/<ref>.json         – BLIP caption

  tok_rgb_next/   <video>/<ref>_n1.npy       – Cosmos tokens of next‑frame #1 …
  coords_next/    <video>/<ref>_n1.npy       – joint tokens of next‑frame #1 …
```
Every RGB token file has a **matching coords file**.
"""

from pathlib import Path
import json
from typing import List, Optional

import numpy as np
import scipy.io
import torch
import torchvision.transforms.functional as TF
from huggingface_hub import snapshot_download
from PIL import Image
from skimage.metrics import structural_similarity as ssim
from tqdm import tqdm
from transformers import BlipForConditionalGeneration, BlipProcessor

# ------------------------ constants ------------------------
TARGET_RES       = 256     # shorter side after resize for Cosmos encoder
SSIM_THRESHOLD   = 0.89   # duplicate‑frame threshold (SSIM > thr ⇒ skip)
REF_PLUS_NEXT    = 7     # 1 reference + 6 distinct successor frames
COORD_QLEVELS    = 8192    # discrete bins for x & y

# ------------------------ similarity ------------------------

def is_similar(img1: Image.Image, img2: Image.Image, thr: float = SSIM_THRESHOLD) -> bool:
    a = np.asarray(img1.convert("L"), dtype=np.float32)
    b = np.asarray(img2.convert("L"), dtype=np.float32)
    return ssim(a, b, data_range=255.0) > thr

# ------------------------ helpers ------------------------

def quantise_coords(x_px: np.ndarray,
                    y_px: np.ndarray,
                    vis:    np.ndarray,
                    img_size: int = TARGET_RES) -> np.ndarray:
    """
    x_px, y_px: pixel coords in [0, img_size]
    vis:        boolean visibility flags
    """
    q = COORD_QLEVELS - 1            # = 8191

    # ---- normalise into [0,1] ----
    x_norm = x_px / img_size
    y_norm = y_px / img_size

    # ---- scale & round into [0…q] ----
    x_tok = np.clip(np.rint(x_norm * q), 0, q).astype(np.uint16)
    y_tok = np.clip(np.rint(y_norm * q), 0, q).astype(np.uint16)

    # visibility stays 0 or 1
    vis_tok = vis.astype(np.uint16)

    return np.stack([x_tok, y_tok, vis_tok])

def write_pair(pair_id: int,
               out_root: Path,
               *,
               ref_coords_tok: np.ndarray,
               next_coords_tok: np.ndarray):
    pid = f"{pair_id:06d}"

    # skip if already exists
    marker = out_root/"coords"/pid/"coords.npy"
    if marker.exists():
        print(f"⏭  Skipping {pid} (already exists)")
        return False

    # make dirs
    for modal in ("coords", "coords_next"):
        (out_root / modal / pid).mkdir(parents=True, exist_ok=True)

    # save
    np.save(out_root/"coords"      /pid/"coords.npy",       ref_coords_tok)
    np.save(out_root/"coords_next" /pid/"coords_next.npy",  next_coords_tok)
    return True

def get_next_available_pair_id(output_root: Path) -> int:
    root = output_root / "coords"
    if not root.exists():
        return 1
    ids = [int(p.name) for p in root.iterdir() if p.is_dir() and p.name.isdigit()]
    return max(ids, default=0) + 1

# ------------------------ core routine ------------------------

def process_video(vid: str,
                  frames_root: Path,
                  labels_root: Path,
                  *,
                  out_root: Path,
                  counter: List[int]):
    frame_dir  = frames_root / vid
    label_path = labels_root / f"{vid}.mat"
    if not frame_dir.exists() or not label_path.exists():
        print(f"⚠️  Skip {vid}: missing data")
        return

    mat          = scipy.io.loadmat(label_path, squeeze_me=True, struct_as_record=False)
    x_all, y_all = mat["x"], mat["y"]
    vis_all      = mat["visibility"].astype(bool)
    T            = int(mat["nframes"])
    H0, W0, _    = mat["dimensions"]
    sx, sy       = TARGET_RES / W0, TARGET_RES / H0

    # load first frame as reference
    frame_files    = sorted(frame_dir.glob("*.jpg"))
    ref_img_full   = Image.open(frame_files[0]).convert("RGB")
    ref_coords_tok = quantise_coords(x_all[0]*sx, y_all[0]*sy, vis_all[0])

    pairs = 0
    for nxt_idx in range(1, T):
        nxt_full = Image.open(frame_files[nxt_idx]).convert("RGB")
        if is_similar(ref_img_full, nxt_full):
            continue

        nxt_coords_tok = quantise_coords(x_all[nxt_idx]*sx,
                                         y_all[nxt_idx]*sy,
                                         vis_all[nxt_idx])

        pid = counter[0]
        write_pair(pid, out_root,
                   ref_coords_tok=ref_coords_tok,
                   next_coords_tok=nxt_coords_tok)
        counter[0] += 1
        pairs += 1

        # shift reference
        ref_img_full   = nxt_full
        ref_coords_tok = nxt_coords_tok

    print(f"✅ {vid}: {pairs} pairs stored")

# ------------------------ wrapper ------------------------

def run(frames_root: Path,
        labels_root: Path,
        output_root: Path,
        *,
        video_start_id: int = 0,
        video_id: Optional[str] = None):
    """
    If `video_id` is given, only that video will be processed.
    Otherwise, all videos from index `video_start_id` onward are processed.
    """
    all_vids = sorted(d.name for d in frames_root.iterdir() if d.is_dir())
    if video_id:
        vids = [video_id] if video_id in all_vids else []
    else:
        vids = all_vids[video_start_id:]

    start_id    = get_next_available_pair_id(output_root)
    pair_counter = [start_id]

    for vid in tqdm(vids, desc="videos"):
        process_video(vid, frames_root, labels_root,
                      out_root=output_root,
                      counter=pair_counter)

    total = pair_counter[0] - start_id
    print(f"🏁 finished: {total} total pairs")


In [ ]:

"""Execute **after** Cell 1.
Edit the three paths and (optionally) `VIDEO_ID`, then run the cell to start
pre‑processing.
"""

# --- user‑editable paths ---------------------------------
FRAMES_ROOT = Path("/home/skalli/COM-304-FM/project/penn_action_raw/Penn_Action/frames")   
LABELS_ROOT = Path("/home/skalli/COM-304-FM/project/penn_action_raw/Penn_Action/labels")  
OUTPUT_ROOT = Path("/home/skalli/COM-304-FM/project/new/output/train")    
# ---------------------------------------------------------

run(
    FRAMES_ROOT,
    LABELS_ROOT,
    OUTPUT_ROOT
)

videos:   0%|                                                                                                               | 1/940 [00:00<14:37,  1.07it/s]

✅ 1387: 0 pairs stored


videos:   0%|▏                                                                                                              | 2/940 [00:01<11:44,  1.33it/s]

✅ 1388: 0 pairs stored


videos:   0%|▎                                                                                                              | 3/940 [00:02<09:38,  1.62it/s]

...

✅ 2324: 18 pairs stored


videos: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 939/940 [17:50<00:01,  1.08s/it]

✅ 2325: 41 pairs stored


videos: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 940/940 [17:52<00:00,  1.14s/it]

✅ 2326: 38 pairs stored
🏁 finished: 13712 total pairs


In [2]:
from pathlib import Path
import shutil
import random

def split_output_dataset(
    output_root: Path,
    train_ratio: float = 0.8,
    eval_ratio: float = 0.1,
    test_ratio: float = 0.1,
    seed: int = 42
):
    assert abs(train_ratio + eval_ratio + test_ratio - 1.0) < 1e-6, "Ratios must sum to 1."

    modalities = ["tok_rgb", "coords", "captions", "tok_rgb_next", "coords_next"]
    random.seed(seed)

    # Find all pair IDs from one modality (they should be the same across all)
    base_dir = output_root / "tok_rgb"
    all_ids = [d.name for d in base_dir.iterdir() if d.is_dir() and d.name.isdigit()]
    all_ids.sort()
    random.shuffle(all_ids)

    N = len(all_ids)
    n_train = int(N * train_ratio)
    n_eval = int(N * eval_ratio)

    splits = {
        "train": all_ids[:n_train],
        "eval":  all_ids[n_train:n_train + n_eval],
        "test":  all_ids[n_train + n_eval:]
    }

    print(f"Found {N} pairs: {len(splits['train'])} train, {len(splits['eval'])} eval, {len(splits['test'])} test")

    for split_name, split_ids in splits.items():
        for pid in split_ids:
            for mod in modalities:
                src = output_root / mod / pid
                dst = output_root / split_name / mod / pid
                if src.exists():
                    dst.parent.mkdir(parents=True, exist_ok=True)
                    shutil.move(str(src), str(dst))

    print("✅ Split completed. Original folders now empty (only train/eval/test remain).")

# Example usage:
split_output_dataset(Path("output"))

Found 41964 pairs: 33571 train, 4196 eval, 4197 test
✅ Split completed. Original folders now empty (only train/eval/test remain).
